## Project 1

* Author: Yanan ZHOU

* Contact: adreambottle@outlook.com

* Topic: Portfolio Construction Method - Thought of Maximum Sharpe Ratio in Combining Different Alphas

### Background

We define alpha as a trading strategy, which can also be considered as a factor, or a feature.

It’s just that this feature, represented by f, represents the forecast of stock returns, and it has some predictive ability, that is, IC = corr(f, r) > 0

Generally speaking, every trading firm will accumulate a lot of such alpha.

Suppose we have N alphas, which means we have N different strategies

Different alphas may make money in different aspects, for example, some are making money from the reversal effect, and some are making money from the momentum effect. How to combine these alphas to make the final generated signal have stronger and more stable predictive power.

This is often a classic regression task in machine learning, i.e. constructing a r_pred = f(a1 + a2 ... aN)

But here I adopt the idea of maximize sharpe ratio to build the alpha combo which is the strongest signal.




### Maximize Sharpe Ratio on different alphas

In the classic Markowitz model, we need to calculate the mean and the correlation for each stock. 

But here we choose to switch dimensions. When we have many strategies, we can calculate the expected return and the covariance matrix of different alphas.

Of course, there are some areas that need modification and attention. For example, when calculating the covariance matrix, how to choose the correct time window, how to make corrections based on the difference between the short term and the long term, and the alpha information generally decays over time, and how to punish the attenuation coefficient.


We can first trade each alpha separately to get their performance on historical backtests.

In this way, we can get a DataFrame. The index of the DataFrame is the trading day, and the columns are each factor. Each item in it is the pnl of a day when a factor is traded alone.

Then record the historical backtest situation of this alpha to calculate the expected return and historical covariance matrix of each alphas.

Then we can perform Maximum Sharpe Ratio Optimization based on the expected rate of return r and the covariance matrix Sigma


In [1]:
# importing libraries

import os
import sys
import bisect
import random
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd

from scipy import optimize
import mosek.fusion as mf

import tushare as ts
token = "xxxxxxxxxxxxxxxxxxxxxx"
ts.set_token(token)
pro = ts.pro_api()

In [2]:
# Functions Needed

def set_random_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

def get_datelist(start: str, end: str, interval: int):
    df = pro.index_daily(ts_code='399300.SZ', start_date=start, end_date=end)
    date_list = list(df.iloc[::-1]['trade_date'])
    sample_list = []
    for i in range(len(date_list)):
        if i % interval == 0:
            sample_list.append(date_list[i])

    return sample_list

def sharpe_func(wts, r, Sigma, decay_ratio):
    if decay_ratio is not None:
        assert len(r) == len(decay_ratio)
        r = r * decay_ratio
    fracNumer = r @ wts
    fracDenom = np.sqrt(wts @ Sigma @ wts.T)
    sharpe = fracNumer / fracDenom
    return sharpe

def revalue_ignore_minimal(x, epsilon=1e-6):
    valid_ = x > epsilon
    x_v = x[valid_]
    x_v = x_v / x_v.sum()
    x_new = np.zeros(x.shape)
    x_new[valid_] = x_v
    return x_new

In [6]:
# Generate pseudo data

set_random_seed()

sta_date = "20210101"
end_date = "20230101"
pred_sta_date = "20220101"

dates_str = np.array(get_datelist(sta_date, end_date, interval=1))
pred_dates_str = dates_str[dates_str >= pred_sta_date]
dates_dt = pd.to_datetime(dates_str)

print(dates_str[:10])
print(dates_dt[:10])

['20210104' '20210105' '20210106' '20210107' '20210108' '20210111'
 '20210112' '20210113' '20210114' '20210115']
DatetimeIndex(['2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13',
               '2021-01-14', '2021-01-15'],
              dtype='datetime64[ns]', freq=None)


In [7]:
nDays = len(dates_dt)
nAlpha = 20
nStock = 3000
alpha_mean = np.random.rand(nAlpha) / 20
alpha_corr = np.random.rand(nAlpha, nAlpha) - 0.5
for i in range(nAlpha):
    for j in range(nAlpha):
        if i < j:
            alpha_corr[i, j] = alpha_corr[j, i]
        elif i == j:
            alpha_corr[i, j] = 1.

decay_ratio = 1 - np.random.rand(nAlpha) * 0.5

alpha_cov = alpha_corr / 100.
df_pnl = np.random.multivariate_normal(alpha_mean, alpha_cov, nDays)
alpha_names = [f"alpha.{i}" for i in range(1, nAlpha+1)]


df_pnl = pd.DataFrame(df_pnl, index=dates_str, columns=alpha_names)
df_pnl.head()

/var/folders/sc/s_x4ss8x4lg2_cd9vrpn5ty80000gn/T/ipykernel_21170/2133621154.py:16: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  df_pnl = np.random.multivariate_normal(alpha_mean, alpha_cov, nDays)


,alpha.1,alpha.2,alpha.3,alpha.4,alpha.5,alpha.6,alpha.7,alpha.8,alpha.9,alpha.10,alpha.11,alpha.12,alpha.13,alpha.14,alpha.15,alpha.16,alpha.17,alpha.18,alpha.19,alpha.20
20210104,0.175507,0.108104,0.097669,0.064396,-0.149448,0.095731,0.232076,0.075986,0.195391,-0.008511,0.040260,0.212479,0.081840,-0.103904,-0.037957,-0.041939,0.171815,0.078638,-0.062942,0.067815
20210105,-0.197140,0.105874,0.037805,0.126539,-0.100943,0.120614,0.096266,0.143533,0.079840,0.003582,-0.024879,0.007868,-0.051814,0.141464,-0.187917,-0.125777,0.057235,0.205519,0.015436,-0.053104
20210106,-0.357126,0.140206,-0.026876,-0.019866,0.066149,0.074192,-0.119102,-0.067050,-0.152019,-0.158415,0.080985,-0.035537,-0.079961,-0.114369,0.064091,0.105191,-0.072333,0.089578,-0.042756,-0.055685
20210107,0.018244,0.120214,-0.045615,0.011167,-0.001188,0.263233,-0.064086,0.196544,0.090425,0.159310,0.122886,0.198280,0.113785,0.022714,0.180259,-0.086065,0.070379,-0.083138,0.104075,-0.122200
20210108,-0.028989,0.211263,0.076148,0.122874,0.167597,0.263187,-0.147169,0.299415,-0.013955,0.047909,-0.202903,0.132695,-0.163651,0.002918,-0.101159,0.205360,-0.110574,-0.096554,0.088282,-0.098233


Because Sharpe Ratio optimization is a fractional optimization problem, not an easy-to-solve convex optimization problem, here I give two solutions, one is to use MOSEK to perform Schaible transform to solve a specific Fraction Optimize Problem, and the other is to use Sequential Least Squares Programming optimize (SLSQP) to solve more general situations, that is, several variables with any combination of bounds, equality and inequality constraints


Solve for the optimal investment portfolio. The weight x maximizes the Sharpe Ratio of the investment portfolio.

$max_{x} \quad \cfrac{r^Tx - r_f}{||Fx||_2} \\ s.t. \quad 1^Tx=1, \\ \quad x \ge 0$​

$x$ is the vector of asset allocations

$F$ is risk associated with the covariance matrix $||Fx||_2 = \sqrt{x^T\Sigma x}$



Because this is fractional programming (FP), and because the numerator is greater than or equal to 0, it is concave, and the denominator greater than 0 is convex. So we can consider concave-convex single-ratio FP

We use the form of Schaible transform to solve

We can set some new variables $w = \cfrac{x}{ \sqrt{x^T\Sigma x}}, \quad t = \cfrac{1}{ \sqrt{x^T\Sigma x}}, \quad w = \cfrac{x}{t}$​



The original maximum Sharpe ratio is equivalent to the convex quadratic problem (QP)

$$max_{x} \quad r^Tw -r_ft \\ s.t. \quad w^T\Sigma w \leq 1, \\ \qquad 1^T w \geq 0,\\ \qquad w \geq 0$$



Then we can eliminate redundant variables t

$$max_{x} \quad w^T(r-r_f\cdot 1) \\ s.t. \quad w^T\Sigma w \leq 1, \\ \qquad 1^T w \geq 0,\\ \qquad w \geq 0$$


Then convert the QP problem to minimize $\sqrt{w^T\Sigma w}$ term

$$min_{x} \quad w^T\Sigma w \\ s.t. \quad w^T(r-r_f\cdot 1)=1, \\ \qquad 1^T w \geq 0,\\ \qquad w \geq 0$$

In this way, the value of Maximize Sharpe Ratio can be obtained very quickly by solving QP.

In [ ]:
class MaxSharpe():

    def __init__(self, r, Sigma, rf, decay_ratio=None, method="MOSEK", tolerance=1e-8):
        """
        r = np.mean(df_pnl, axis=0) # mean return of all alphas
        rf = 0. # risk free interest rate
        Sigma = np.cov(df_pnl, rowvar=False) # covariance of returns of all alphas
        """
        self.r = r
        self.rf = rf
        self.Sigma = Sigma
        self.tolerance = tolerance
        self.nAlpha = len(r)
        self.method = method.lower()

    def solve(self):
        if self.method == "mosek":
            print("Call MOSEK to solve the problem")
            r = self.r + 1.
            rf = self.rf + 1.
            Sigma = self.Sigma
            if not self.is_pos_def(Sigma):
                II = np.eye(self.nAlpha) * 1e-10
                Sigma = Sigma + II
            L_matrix = np.linalg.cholesky(Sigma)
            F = L_matrix.T

            try:
                wts = self.MaxSharpeMosek(r, rf, F, full=True, pos=True)
            except:
                print("MOSEK Fails, using SLSQP method")
                r = self.r
                rf = self.rf
                Sigma = self.Sigma
                tolerance = self.tolerance
                wts = self.MaxSharpeSLSQP(r, rf, Sigma, tolerance)

        if self.method == "slsqp":
            r = self.r
            rf = self.rf
            Sigma = self.Sigma
            tolerance = self.tolerance
            wts = self.MaxSharpeSLSQP(r, rf, Sigma, tolerance)

        return wts
    
    def MaxSharpeMosek(self, r, rf, F, full=True, pos=True):
        with mf.Model("Sharpe") as M:
            n = self.nAlpha
            y = M.variable("y", n. mf.Domain.greaterThan(0) if pos else mf.Domain.unbounded())
            z = M.variable("z", mf.Domain.greaterThan(0))

            t = M.variable()
            M.constraint(mf.Expr.vstack(t, mf.Expr.mul(F, y)), mf.Domain.inQCone())
            M.objective(mf.ObjectiveSense.Minimize, t)

            M.constraint(mf.Expr.sub(mf.Expr.dot(r, y), mf.Expr.mul(rf, z)), mf.Domain.equalsTo(1))

            if full:
                M.constraint(mf.Expr.sub(mf.Expr.sum(y), z), mf.Domain.equalsTo(0))

            M.solve()

            if M.getProblemStatus() == mf.ProblemStatus.PrimalAndDualFeasible and z.level()[0] > 0:
                zval = z.level()[0]
                return np.array([yi/zval for yi in y.level()])
            else:
                raise ValueError("No Solution or some issue")


    def MaxSharpeSLSQP(self, r, rf, Sigma, tolerance=1e-7):
        
        def neg_sharpe_func(x, r, Sigma, rf):
            fracNumer = r @ x.T - rf
            fracDenom = np.sqrt(x @ Sigma @ x.T)
            sharpe = fracNumer / fracDenom
            return -sharpe

        def constraintEq(x):
            A = np.ones(x.shape)
            b = 1
            constraintVal = np.matmul(A, x.T) - b
            return constraintVal
        
        nAlpha = self.nAlpha
        x_init = np.repeat(1/nAlpha, nAlpha)
        constraints = ({"type" : "eq", "fun":constraintEq})
        lb = 0.
        ub = 1.
        bounds = tuple([(lb, ub) for xi in x_init])

        opt = optimize.minimize(
            neg_sharpe_func,
            x0 = x_init,
            args = (r, Sigma, rf, nAlpha),
            method = "SLSQP",
            bounds = bounds,
            constraints = constraints,
            tol = tolerance
        )
        wts = opt.x
        return wts


def get_datelist(start: str, end: str, interval: int):
    df = pro.index_daily(ts_code='399300.SZ', start_date=start, end_date=end)
    date_list = list(df.iloc[::-1]['trade_date'])
    sample_list = []
    for i in range(len(date_list)):
        if i % interval == 0:
            sample_list.append(date_list[i])

    return sample_list

### Full Code

Here are the full codes of this problem, for more see the "max_sharpe.py" filw

In [ ]:
def main():

    sta_date = "20210101"
    end_date = "20230101"
    pred_sta_date = "20220101"
    
    dates_str = np.array(get_datelist(sta_date, end_date, interval=1))
    pred_dates_str = dates_str[dates_str >= pred_sta_date]
    dates_dt = pd.to_datetime(dates_str)
    
    nDays = len(dates_dt)

    nAlpha = 20
    nStock = 3000
    alpha_mean = np.random.rand(nAlpha) / 20
    alpha_corr = np.random.rand(nAlpha, nAlpha) - 0.5
    for i in range(nAlpha):
        for j in range(nAlpha):
            if i < j:
                alpha_corr[i, j] = alpha_corr[j, i]
            elif i == j:
                alpha_corr[i, j] = 1.
    
    decay_ratio = 1 - np.random.rand(nAlpha) * 0.5

    alpha_cov = alpha_corr / 100.
    df_pnl = np.random.multivariate_normal(alpha_mean, alpha_cov, nDays)
    alpha_names = [f"alpha.{i}" for i in range(1, nAlpha+1)]
    

    df_pnl = pd.DataFrame(df_pnl, index=dates_str, columns=alpha_names)
    
    rf = 0.03 / 365
    rolling = False
    rolling_window = 180
    mean_decay = True
    cov_decay = False

    wts_list = []
    for tidx, today in tqdm(enumerate(pred_dates_str)):
        if rolling:
            train_index = dates_str[dates_str < today]
            train_len = len(train_index)
        else:
            train_index = dates_str[dates_str < today]
            train_len = len(train_index)
            
            train_index = train_index[train_len-rolling_window:train_len]
        
        df_pnl_sep = df_pnl.loc[train_index, :]
        decay_ratio = np.repeat(decay_ratio, train_len).reshape(nAlpha, -1).T
        df_pnl_sep_decay = df_pnl_sep * decay_ratio
        if mean_decay:
            r = df_pnl_sep_decay.means().values
        else:
            r = df_pnl_sep.means().values
        
        if cov_decay:
            Sigma = df_pnl_sep_decay.cov().values
        else:
            Sigma = df_pnl_sep.cov().values

        wts = MaxSharpe(r, Sigma, rf=rf, method="MOSEK").solve()
        wts_list.append(wts)

        fcst = np.random.randn(nStock, nAlpha)
        fcst = pd.DataFrame(fcst, columns=alpha_names)
        combo_fcst = fcst @ wts 
        combo_fcst.to_csv(f"{today}_combo_fcst.csv")